In [3]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_columns = 5000
%matplotlib inline

dft = pd.read_csv('data/train.csv')

dft = dft.drop('Id', axis = 1)

In [4]:
Y = dft['SalePrice']
dft = dft.drop('SalePrice', axis=1)

In [5]:
cont = []
for i in dft.columns:
    if (dft[i].dtype == 'float64') | (dft[i].dtype == 'int64'):
        cont.append(i)

        
YrMo = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'MoSold', 'YrSold']

for i in YrMo:
    cont.remove(i)

dft[cont] = dft[cont].astype('float64')


dates = dft[YrMo]

In [6]:
x = {}
for i in cont:
    x[i] = dft[i].nunique()

x = pd.DataFrame.from_dict(x, orient='index')

y = ['LowQualFinSF', '3SsnPorch', 'PoolArea']

mode_na = ['Alley', 'PoolQC', 'Fence','MiscFeature', 'FireplaceQu']


nums = dft[list(x[x.iloc[:,0]>=50].index)].astype(object)
num_cats = dft[list(x[x.iloc[:,0]<50].index)].astype(object)

to_drop = list(nums.columns)+list(dates.columns)+list(num_cats.columns)+mode_na
cats =  dft.drop(to_drop, axis = 1)
cats_na = dft[mode_na]

dates = dft[YrMo].astype(object)


#move continuous variables with low-frequency from num_cats to nums
nums[y] = num_cats[y]
num_cats = num_cats.drop(y, axis=1)

In [7]:
nums = nums.fillna(0)

In [8]:
def add_dummies_remove_modes(dummy_list, house_raw, house_df):

    dummy_modes = list(house_raw[dummy_list].mode().iloc[0,:].items())

    dummy_modes = [(col, (float(mode))) if type(mode) == int else (col, mode) for col, mode in dummy_modes]

    drop_modes = list(map(lambda x: str(x[0]) + '_' + str(x[1]), dummy_modes))

    dummy_cols = pd.get_dummies(house_df[dummy_list].astype(object), dummy_na=True).drop(drop_modes, axis=1)

    return pd.concat([
       house_df.drop(dummy_list, axis=1),
       dummy_cols], axis=1)


In [9]:
#num_cats dummies
num_cats = num_cats.fillna(num_cats.mode())
x = add_dummies_remove_modes(list(num_cats.columns), dft, num_cats)

x = x.T.drop_duplicates(keep='first').T

x = x.drop(x.columns[x.sum()==0], axis=1)

num_cats = x

In [10]:
#cats dummies
x = add_dummies_remove_modes(list(cats.columns),dft,cats)

x = x.T.drop_duplicates(keep='first').T

x = x.drop(x.columns[x.sum()==0], axis=1)

cats = x

In [11]:
x = pd.get_dummies(cats_na)

x = x.T.drop_duplicates(keep='first').T

x = x.drop(x.columns[x.sum()==0], axis=1)

cats_na = x

In [12]:
dates['GarageBlt'] = ~dates.GarageYrBlt.isna()
dates = dates.drop('GarageYrBlt', axis=1)
dates['GarageBlt'] = dates['GarageBlt'].apply(lambda x: sum([x]))

In [13]:
df_train_final = pd.concat([dates, nums, num_cats, cats, cats_na], axis=1)

In [14]:
df_train_final.shape

(1460, 330)

In [15]:
df_train_final = df_train_final.T.drop_duplicates(keep='first').T

(1460, 327)

In [ ]:
df_train_final.to_pickle('C:/Users/phili/NYCDSA/Machine_Learning/Project1/kaggle-housing-prices/phil_train_clean_pipe.pkl')

In [ ]:
# from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

# lasso  = Lasso()
# alphas = np.linspace(0.1,20.1,20)
# lasso.set_params(normalize=False)
# coefs_lasso  = []

# for alpha in alphas:
#         lasso.set_params(alpha=alpha)
#         lasso.fit(dfpd, Y)  
#         coefs_lasso.append(lasso.coef_)

# coefs_lasso = pd.DataFrame(coefs_lasso, index = alphas, columns = dfpd.columns)  
# coefs_lasso.head()